# Elimination

In our Elimination process, we have two different type of analysis: removal of duplicates and removal ofplanets with controversial flags of 1. 

## Deduplication

The NASA Exoplanet Archive has over 35,000 entries as of July 2022, consisting of duplicate values for the majority of planets listed. Thus, to eliminate the various duplicate entries, we initially read the dataset as a Pandas dataframe via the Pandas module in Python. After, we arranged all the entries based on the publication date of each planet in ascending order - which can be denoted as disc_pubdate. The sorting of data was crucial to preserving the most recent and accurate finding for each exoplanet, as we only intend on keeping the last occurrence of a particular exoplanet, since it is assumed to be the most up-to-date. The removal of duplicate entries in NASA’s Exoplanet Archive successfully narrows down 35,000 entries to simply 5,055 exoplanets. Thus, it yields an elimination of almost 30,000 planets standalone - making it a vital process when performing our analysis. 


In [1]:
import pandas as pd
import numpy as np
import csv

df = pd.read_csv('../data/dataset.csv', low_memory=False)

df.sort_values('disc_pubdate', ascending=True)

df = df.drop_duplicates(subset=['pl_name'], keep='last')

df.reset_index(drop=True, inplace=True)



## Removal of controversial flags


After wiping out entries of duplicate exoplanets with  outdated publication dates, we ventured to further our process of elimination by removing entries with a controversial flag of 1. Controversial flags are denoted by the term pl_controv_flag, by which they are represented in boolean values of either 0 or 1. When an exoplanet has a controversial flag of 0, then it is considered to be a confirmed planet, making it a potential contender for being capable of sustaining human life. However, if an exoplanet has a controversial flag of 1, then its existence has been questioned in the past - making it not completely confirmed. Furthermore, our  

In [2]:
df = df.loc[df['pl_controv_flag'] != 1]

df.reset_index(drop=True, inplace=True)

# CHZ

Through our CHZ process, we eliminate all the planets that are not within our circumstellar habitable zone. Amid our process, we consider various factors such as the stellar type, planetary luminocity, bolometric lumniocity, absolute magnitude, semi minor/major axis, and more. 

## Luminocity and absolute magnitude

After de-duplicating the dataset and removing planets with controversial flags, we were ready to perform the CHZ calculations. The stellar luminosity was the core of these calculations because it was needed to determine the inner and outer bounds of the CHZ. 

The values for stellar luminosity in the dataset were represented as logarithms of their actual values, so if there was a value for the luminosity of the host star of an exoplanet, we had to raise 10 to the power of that value to determine the host star’s luminosity (in solar luminosities), as shown in Figure 1.

While performing this calculation, we found that many planets had null values for their host star’s luminosity. Therefore, we used the Stefan-Boltzmann law to calculate the luminosity of a host star if it did not already have a value for its luminosity. However, the Stefan-Boltzmann law required values for stellar radius and stellar temperature, so if a host star’s luminosity and at least one of its radius and temperature values were null, there was no possible way to determine its luminosity. There were 594 exoplanets for which the aforementioned conditions were true, so we had to remove them from the dataset. However, this does not mean that the removed exoplanets are not habitable; we simply could not determine their habitability because they were missing too many values in the dataset. 

In [3]:
for i in range(0, len(df)):

    # The values for luminosity in the dataset are represented as logs, so
    # if there is a value, the actual luminosity (in solar luminosities)
    # is 10 to the power of that value

    if pd.notna(df['st_lum'][i]):
        df['st_lum'][i] = 10**(df['st_lum'][i])

    # If there's not a luminosity value, but there are stellar radius and
    # temperature values, the luminosity can be calculated using the
    # Stefan-Boltzmann law, with all untis in terms of the Sun's physical
    # parameters (the Sun's temperature is 5778 K and the stellar radius is
    # already in terms of the Sun's radius)

    if ((pd.isna(df['st_lum'][i])) & ((pd.notna(df['st_teff'][i])) & (pd.notna(df['st_rad'][i])))):
        df['st_lum'][i] = (df['st_rad'][i]**2) * ((df['st_teff'][i]/5778)**4)

    # Otherwise, at least the luminosity and at least one of the stellar radius
    # and stellar temperature are missing, so it is impossible to calculate
    # the star's luminosity

    else:
        pass

# After the luminosity calculations have been performed,
# planets with no values for their star's luminosity must be removed

df = df.loc[pd.notna(df['st_lum'])]

df.reset_index(drop=True, inplace=True)

# Creating a new column for absolute magnitude, based on the relation
# between absolute magnitude and luminosity (see Wikipedia page)

df['st_abs_mag'] = 4.74 + (-2.5*np.log10(df['st_lum']))

C:\Users\shobh\AppData\Local\Temp\ipykernel_14320\1353617133.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['st_lum'][i] = 10**(df['st_lum'][i])
C:\Users\shobh\AppData\Local\Temp\ipykernel_14320\1353617133.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['st_lum'][i] = (df['st_rad'][i]**2) * ((df['st_teff'][i]/5778)**4)


## Calculation of Bolometric Magnitude and Luminosity Based on Stellar Temperature / Spectral Class

The inner and outer bounds of the CHZ depend on the bolometric luminosity, not just standard stellar luminosity. Bolometric luminosity is a star’s luminosity across all wavelengths, as opposed to a star’s luminosity when viewed in a particular band. The definition for bolometric magnitude follows similarly. The absolute magnitude of each star determined above was the visual absolute magnitude. To determine the bolometric magnitude and then the bolometric luminosity, a bolometric correction must be added to the visual absolute magnitude. The bolometric correction depends on a star’s spectral class (or temperature). Once we determined the bolometric magnitude for each host star, we were able to calculate the bolometric luminosity of each host star using a rearranged form of Equation 1, as shown below. 

In [4]:
df['st_spectype'] = df['st_spectype'].astype(str)

df['bol_mag'] = 0

# The for loop below adds a bolometric magnitude for the host star of each planet
# BC is the bolometric correction, which is used to convert from absolute magnitude
# (st_abs_mag) to bolometric magnitude

for i in range(0, len(df)):
    BC = 0
    if (df['st_teff'][i] >= 2400 and df['st_teff'][i] <= 3700) or df['st_spectype'][i][0] == 'M':
        BC = -2.0
    elif (df['st_teff'][i] >= 3700 and df['st_teff'][i] <= 5200) or df['st_spectype'][i][0] == 'K':
        BC = -0.8
    elif (df['st_teff'][i] >= 5200 and df['st_teff'][i] <= 6000) or df['st_spectype'][i][0] == 'G':
        BC = -0.4
    elif (df['st_teff'][i] >= 6000 and df['st_teff'][i] <= 7500) or df['st_spectype'][i][0] == 'F':
        BC = -0.15
    elif (df['st_teff'][i] >= 7500 and df['st_teff'][i] <= 10000) or df['st_spectype'][i][0] == 'A':
        BC = -0.3
    elif (df['st_teff'][i] >= 10000 and df['st_teff'][i] <= 30000) or df['st_spectype'][i][0] == 'B':
        BC = -2.0
    df['bol_mag'][i] = df['st_abs_mag'][i] + BC

# Creating a new column for BOLOMETRIC luminosity, based on BOLOMETRIC Magnitude
# (This is different from st_lum and st_abs_mag)
# The bolometric luminosity must be used for the CHZ calculations
# Once again, this formula can be found on Wikipedia

df['bol_lum'] = 10**((df['bol_mag']-4.74)/-2.5)



C:\Users\shobh\AppData\Local\Temp\ipykernel_14320\4064999246.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bol_mag'][i] = df['st_abs_mag'][i] + BC


## Determining habitability

Once we calculated the bolometric luminosity for each host star, we were able to determine the inner and outer bounds of the CHZ, in AU, for each host star using Equations 2 and 3 below (taken from Tom Morris’s paper). 

We then checked if the exoplanet’s average orbiting distance around its host star was within the CHZ bounds of its host star. The average orbiting distance is dependent on the eccentricity of the orbit; if the eccentricity is small, the semi-major axis is a good approximation of the average orbiting distance, but if the eccentricity is large, the approximation breaks down. We decided that if the eccentricity of an exoplanet’s orbit was less than or equal to 0.2, the semi-major axis would be a good approximation of the exoplanet’s average orbiting distance. If the eccentricity was greater than 0.2, we used Equation 3 for the average orbiting distance, in which dₐᵥ is the mean distance averaged over time, a is the semi-major axis, and e is the eccentricity. If there was no value for the semi-major axis of a planet’s orbit, but there was a value for its orbital period, we used Kepler’s 3rd Law (Equation 5) to determine the semi-major axis, where T was the orbital period in years, a was the semi-major axis in AU, and M was the mass of the host star in solar masses. If the average orbiting distance of an exoplanet was within the CHZ bounds of its host star, we considered it to be habitable. If the eccentricity value for an exoplanet was null or the values for its semi-major axis, orbital period, and host star’s mass were null, we could not determine the habitability of the exoplanet. 

In [5]:
# Defining the inner boundary of the CHZ (formula from Tom Morris)

df['inner_CHZ'] = np.sqrt(df['bol_lum']/1.1)

# Defining the outer boundary of the CHZ (formula from Tom Morris)

df['outer_CHZ'] = np.sqrt(df['bol_lum']/0.53)

# Creating a new column to determine if planets are habitable

df['Habitable'] = None

planets = []

for i in range(0, len(df)):

    # First if statement checks if there are values for the planet's
    # semi-major axis and eccentricity

    if ((pd.notna(df['pl_orbsmax'][i])) & (pd.notna(df['pl_orbeccen'][i]))):

        # If the eccentricity is less than 0.2, the average distance from the planet
        # to its star can be approximated by its semi-major axis
        # If the average distance is in the CHZ range, we say the planet is
        # habitable

        if df['pl_orbeccen'][i] <= 0.2:
            if ((df['pl_orbsmax'][i] >= df['inner_CHZ'][i]) & (df['pl_orbsmax'][i] <= df['outer_CHZ'][i])):
                df['Habitable'][i] = 'Yes'
                planets.append(df['pl_name'][i])
            else:
                df['Habitable'][i] = 'No'

        # If the eccentricity is greater than 0.2, we must use a formula for
        # the average distance. TODO: CITE SOURCE!! . and then check if this
        # average distance is in the CHZ range

        else:
            avg_dist = df['pl_orbsmax'][i] * (1+((df['pl_orbeccen'][i]**2)/2))
            if ((avg_dist >= df['inner_CHZ'][i]) & (avg_dist <= df['outer_CHZ'][i])):
                df['Habitable'][i] = 'Yes'
                planets.append(df['pl_name'][i])
            else:
                df['Habitable'][i] = 'No'

    # If there is no value for the planet's semi-major axis, but there are values
    # for its orbital period, eccentricity, and the star's mass, we can use
    # Kepler's 3rd Law to calculate its semi-major axis, and then perform
    # the eccentricity check again to determine the average orbiting distance
    # and then determine if this average distance is in the CHZ

    elif ((pd.isna(df['pl_orbsmax'][i])) & (pd.notna(df['pl_orbeccen'][i])) & (pd.notna(df['pl_orbper'][i])) & (pd.notna(df['st_mass'][i]))):

        # Convering the orbital period from days to years

        orb_per_year = df['pl_orbper'][i]/365

        # Using Kepler's Third Law: T^2/a^3 = 1/M, with T in years, a in AU
        # and M in solar masses, so a = cubert(T^2 * M)

        df['pl_orbsmax'][i] = np.cbrt((orb_per_year**2)*df['st_mass'][i])

        if df['pl_orbeccen'][i] <= 0.2:
            if ((df['pl_orbsmax'][i] >= df['inner_CHZ'][i]) & (df['pl_orbsmax'][i] <= df['outer_CHZ'][i])):
                df['Habitable'][i] = 'Yes'
                planets.append(df['pl_name'][i])
            else:
                df['Habitable'][i] = 'No'

        else:
            avg_dist = df['pl_orbsmax'][i] * (1+((df['pl_orbeccen'][i]**2)/2))
            if ((avg_dist >= df['inner_CHZ'][i]) & (avg_dist <= df['outer_CHZ'][i])):
                df['Habitable'][i] = 'Yes'
                planets.append(df['pl_name'][i])
            else:
                df['Habitable'][i] = 'No'

    # Otherwise, there are too many values missing to determine if the planet
    # is habitable. CND = Can Not Determine

    else:
        df['Habitable'][i] = 'CND'

# print(planets, len(planets))

C:\Users\shobh\AppData\Local\Temp\ipykernel_14320\4027985643.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Habitable'][i] = 'No'
C:\Users\shobh\AppData\Local\Temp\ipykernel_14320\4027985643.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Habitable'][i] = 'No'
C:\Users\shobh\AppData\Local\Temp\ipykernel_14320\4027985643.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Habitable'][i] = 'CND'
C:\Users\shobh\App

BD-06 1339 c
GJ 3293 c
GJ 3323 c
GJ 876 c
HD 10180 g
HD 103891 b
HD 10442 b
HD 10697 b
HD 109286 b
HD 114729 b
HD 125612 b
HD 128356 b
HD 132406 b
HD 141937 b
HD 142415 b
HD 147513 b
HD 159868 b
HD 16175 b
HD 163607 c
HD 165155 b
HD 17674 b
HD 18015 b
HD 181720 b
HD 183263 b
HD 190647 b
HD 213240 b
HD 214823 b
HD 216437 b
HD 219415 b
HD 221585 b
HD 222582 b
HD 224538 b
HD 23079 b
HD 23127 b
HD 34445 b
HD 38529 c
HD 4113 b
HD 43197 b
HD 65216 b
HD 7199 b
HD 9174 b
HIP 5158 b
KELT-6 c
KIC 9663113 b
Kapteyn c
Kepler-1318 b
Kepler-16 b
Kepler-1600 b
Kepler-1630 b
Kepler-1647 b
Kepler-1649 c
Kepler-1704 b
Kepler-1708 b
Kepler-174 d
Kepler-1868 b
Kepler-440 b
Kepler-454 c
NY Vir c
WASP-41 c
WASP-47 c
Wolf 1061 d


# Visualizing our data

# Exporting our data

In [ ]:
for planet in planets:
    print(planet)

with open('../data/habitable-planets.csv', 'w', newline='') as csvfile:
    fieldnames = ['number', 'pl_name']
    planet_count = 0

    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for planet in planets:
        planet_count += 1
        writer.writerow({'number': planet_count, 'pl_name': planet})